In [1]:
include("../../BFFPSV18.jl")

add_time! (generic function with 1 method)

## Model

In [ ]:
# ===================================
# Model: Motor (8 variables, 2 inputs)
# ===================================

using SparseArrays

# =====================
# Problem specification
# =====================
I = [1, 2, 2, 3, 3, 3, 3, 4, 5, 6, 6, 7, 7, 7, 7, 8]
J = [2, 3, 2, 1, 2, 3, 4, 1, 6, 7, 6, 5, 6, 7, 8, 5]
vals = [1, 8487.2, -1.0865, -2592.1, -21.119, -698.91, -141399.0, 1.0, 1.0,
       8487.2, -1.0865, -2592.1, -21.119, -698.91, -141399.0, 1.0]
A = sparse(I, J, vals)

# initial set
X0 = Hyperrectangle([0.00225, 0.0, 0.0, 0.0, 0.00125, 0.0, 0.0, 0.0],
                    [0.00025, 0.0, 0.0, 0.0, 0.00025, 0.0, 0.0, 0.0])

# input set
B = sparse([4, 8], [1, 2], [-1.0, -1.0])
U = Hyperrectangle([0.23, 0.3], [0.07, 0.1])

# instantiate continuous LTI system
S = ConstrainedLinearControlContinuousSystem(A, B, nothing, U)
P = InitialValueProblem(S, X0)

## Reach

### Solution with upstream implementation

In [ ]:
# uniform, 2D partition
partition_2D = [(2*i-1:2*i) for i in 1:4] # 2D blocks

In [ ]:
opts = Options(:T => 20.0, :plot_vars => [0, 5])
opts_algo = Options(:δ => 0.001, :vars => [5], :partition=>partition_2D)

In [ ]:
sol = solve(P, opts, op=BFFPSV18(opts_algo));

In [ ]:
using BenchmarkTools

In [ ]:
@btime solve($P, $opts, op=BFFPSV18($opts_algo));

In [ ]:
using Plots

In [ ]:
# plot(sol, xlab="t", ylab="x5") # uncomment to plot (Jupyter notebook gets slowed down with large figures)

### Solution with revised implementation

In [ ]:
S = LinearContinuousSystem(MMatrix{8, 8, Float64}(A))
P = InitialValueProblem(S, X0)

# UNIFORM 1D partition
opts = Options(:T => 20.0,
               :δ => 0.001,
               :N => 20_000,  # el N se puede sacar de T y de delta
               :vars=>[5],
               :partition => [[1], [2], [3], [4], [5], [6], [7], [8]],
               :set_type => Interval{Float64, IA.Interval{Float64}},
               :row_blocks => [[5]],
               :column_blocks => [[1], [2], [3], [4], [5], [6], [7], [8]],
               :num_type => Float64,
               :block_indices => [5]);
# varias de las opciones anteriores se pueden deducir de otras!

In [ ]:
sol = solve_BFFPSV18(P, opts);

In [ ]:
@btime solve_BFFPSV18($P, $opts);
#

# some results:
# homogeneous case (no input)  85.737 ms (100333 allocations: 4.30 MiB)
#
#

In [ ]:
set(sol[1])

In [ ]:
# add time as the first variable
add_time!(sol)

In [ ]:
set(sol[1])

In [ ]:
# slow (there are 20 thousand sets to plot)
# plot([set(sol[i]) for i in eachindex(sol)], xlab="t", ylab="x5")

## Check

In [ ]:
# property: x1 < 0.35 || x5 < 0.45
property = Disjunction([SafeStatesProperty(HalfSpace([1.; zeros(7)], 0.35)),
                        SafeStatesProperty(HalfSpace([zeros(4); 1.; zeros(3)], 0.45))])

In [ ]:
opts = Options(:plot_vars => [0, 5], :property => property)
opts_algo = Options(:vars => [5], :partition => partition_2D)

# Results

```julia
julia> versioninfo()

Julia Version 1.2.0
Commit c6da87ff4b (2019-08-20 00:03 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-8705G CPU @ 3.10GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)
```


## Reach algorithm

### All variables


|Algorithm|Partition|Set type|Runtime (ms) and Allocations|Speedup|
|------|-----------|------|------|------|
|upstream|2d uni|hyperrectangle| 895.893 ms (2000539 allocations: 106.88 MiB)|1x|
|upstream|1d uni|interval|||
|local|2d uni|hyperrectangle|||
|local|1d uni| interval|||
|local, w/sarray|1d uni| interval|||

The speedup is measured wrt the fastest upstream method, in this case interval.

### One variable

## Check algorithm